In [1]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, ClassifierMixin

import lightgbm as lgb

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
class VotingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

In [3]:

# pre-processing functions
def drop_high_null_columns(df, threshold=0.5):
    # Calculate the percentage of null values for each column
    null_percentages = df.isnull().mean()
    # Identify columns where the percentage of null values exceeds the threshold
    cols_to_drop = null_percentages[null_percentages > threshold].index.tolist()
    # Drop the identified columns from the DataFrame
    df = df.drop(columns=cols_to_drop)
    return df

# def drop_high_cardinality_columns(df, threshold=0.9): 
#     # Identify columns that have a high number of unique values
#     high_cardinality_cols = []
#     for col in df.columns:
#         if df[col].nunique() / df.shape[0] > threshold:
#             high_cardinality_cols.append(col)
#     # Drop the identified columns from the DataFrame
#     df = df.drop(columns=high_cardinality_cols)
#     return df

In [4]:
class Pipeline:
    @staticmethod
    def set_table_dtypes(df):
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))            

        return df
    
    @staticmethod
    def handle_dates(df):
        for col in df.columns:
            if col[-1] in ("D",):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))
                df = df.with_columns(pl.col(col).dt.total_days())
                df = df.with_columns(pl.col(col).cast(pl.Float32))
                
        df = df.drop("date_decision", "MONTH")

        return df
    
    @staticmethod
    def filter_cols(df):
        # List of attributes to exclude

        # manual select the columns to remove
       excluded_attributes = [
    "addres_district_368M", "addres_role_871L", "addres_zip_823M",
    "amtinstpaidbefduel24m_4187115A", "annualeffectiverate_199L", "annuity_853A",
    "applicationcnt_361L", "applications30d_658L", "applicationscnt_1086L",
    "applicationscnt_464L", "applicationscnt_629L", "applicationscnt_867L",
    "approvaldate_319D", "assignmentdate_238D", "assignmentdate_4527235D",
    "assignmentdate_4955616D", "bankacctype_710L", "birth_259D", "birthdate_574D",
    "birthdate_87D", "byoccupationinc_3656910L", "cardtype_51L", "childnum_21L",
    "classificationofcontr_1114M", "classificationofcontr_13M", "classificationofcontr_400M",
    "clientscnt_100L", "clientscnt_1022L", "clientscnt_1071L", "clientscnt_1130L",
    "clientscnt_136L", "clientscnt_157L", "clientscnt_257L", "clientscnt_304L",
    "clientscnt_360L", "clientscnt_493L", "clientscnt_533L", "clientscnt_887L",
    "clientscnt_946L", "clientscnt12m_3712952L", "clientscnt3m_3712950L",
    "clientscnt6m_3712949L", "cntpmts24_3658933L", "collater_typofvalofguarant_298M",
    "collater_typofvalofguarant_407M", "collaterals_typeofguarante_359M",
    "collaterals_typeofguarante_669M", "contaddr_district_15M", "contaddr_matchlist_1032L",
    "contaddr_smempladdr_334L", "contaddr_zipcode_807M", "contractdate_551D",
    "contractenddate_991D", "contractmaturitydate_151D", "contractst_964M",
    "contractsum_5085717L", "contracttype_653M", "conts_role_79M", "conts_type_509L",
    "creationdate_885D", "credlmt_228A", "credlmt_230A", "credor_3940957M", "credtype_322L",
    "credtype_587L", "dateactivated_425D", "datefirstoffer_1144D", "datelastinstal40dpd_247D",
    "datelastunpaid_3546854D", "dateofcredend_289D", "dateofcredend_353D", "dateofcredstart_181D",
    "dateofcredstart_739D", "dateofrealrepmt_138D", "deductiondate_4917603D", "description_351M",
    "description_5085714M", "disbursementtype_67L", "district_544M", "dpdmaxdatemonth_804T",
    "dpdmaxdatemonth_89T", "dpdmaxdateyear_596T", "dpdmaxdateyear_742T", "dpdmaxdateyear_896T",
    "dtlastpmt_581D", "dtlastpmtallstes_3545839D", "dtlastpmtallstes_4499206D", "education_1103M",
    "education_1138M", "education_88M", "education_927M", "eir_270L", "empl_employedfrom_271D",
    "empladdr_district_926M", "empladdr_zipcode_114M", "employedfrom_700D", "employername_160M",
    "empls_employedfrom_796D", "empls_employer_name_740M", "familystate_447L", "familystate_726L",
    "financialinstitution_382M", "financialinstitution_591M", "firstclxcampaign_1125D",
    "firstdatedue_489D", "firstnonzeroinstldate_307D", "fourthquarter_440L", "gender_992L",
    "housetype_905L", "housingtype_772L", "incometype_1044T", "inittransactioncode_186L",
    "inittransactioncode_279L", "isbidproduct_1095L", "isbidproduct_390L", "isbidproductrequest_292L",
    "isdebitcard_527L", "isdebitcard_729L", "language1_981M", "last180dayturnover_1134A",
    "last30dayturnover_651A", "lastactivateddate_801D", "lastapplicationdate_877D",
    "lastapprcommoditycat_1041M", "lastapprcommoditytypec_5251766M", "lastapprdate_640D",
    "lastcancelreason_561M", "lastdelinqdate_224D", "lastrejectcommoditycat_161M",
    "lastrejectcommodtypec_5251769M", "lastrejectdate_50D", "lastrejectreason_759M",
    "lastrejectreasonclient_4145040M", "lastrepayingdate_696D", "lastupdate_1112D",
    "lastupdate_260D", "lastupdate_388D", "maritalst_385M", "maritalst_703L", "maritalst_893M",
    "maxdpdinstldate_3546855D", "mobilephncnt_593L", "name_4527232M", "name_4917606M",
    "numberofoverdueinstlmaxdat_148D", "numberofoverdueinstlmaxdat_641D", "openingdate_313D",
    "openingdate_857D", "overdueamountmax2date_1002D", "overdueamountmax2date_1142D",
    "overdueamountmaxdatemonth_284T", "overdueamountmaxdatemonth_365T", "overdueamountmaxdatemonth_494T",
    "overdueamountmaxdateyear_2T", "overdueamountmaxdateyear_432T", "overdueamountmaxdateyear_994T",
    "paytype_783L", "paytype1st_925L", "payvacationpostpone_4187118D", "periodicityofpmts_1102L",
    "periodicityofpmts_837L", "personindex_1023L", "persontype_1072L", "persontype_792
       ]


        # Exclude the attributes
        df = df.drop(excluded_attributes, errors='ignore')

        # Preprocess the dataframe
        df = drop_high_null_columns(df)
        # df = drop_high_cardinality_columns(df)
  
        # Print the null values for each column
        for col in df.columns:
            null_count = df[col].isnull().sum()  # Count the number of null values
            print(f"Null values in '{col}': {null_count}")

       # Impute missing values
        for col in df.columns:
            if df[col].dtype == 'object' or df[col].dtype == 'category':
                if not df[col].mode().empty:
                    mode = df[col].mode().iloc[0]  # Calculate the mode
                    df[col].fillna(mode, inplace=True)  # Fill null values with the mode inplace
                    print(f"Imputed missing values in '{col}' with mode ('{mode}').")
                else:
                    print(f"No mode available for column '{col}'. Skipping imputation for this column.")
            elif df[col].dtype in ['int64', 'float64']:
                # Check the skewness of the column
                skewness = df[col].skew()
                if skewness < -1 or skewness > 1:  # Highly skewed
                    median = df[col].median()
                    df[col].fillna(median, inplace=True)
                    print(f"Imputed missing values in '{col}' with median ({median}) due to high skewness ({skewness}).")
                else:  # Normal or slightly skewed
                    mean = df[col].mean()
                    df[col].fillna(mean, inplace=True)
                    print(f"Imputed missing values in '{col}' with mean ({mean}).")
    

        # Print the null values for each column after imputation
        for col in df.columns:
            null_count = df[col].isnull().sum()  # Count the number of null values
            print(f"Null values in '{col}' after imputation: {null_count}")

        # Print the number of columns left
        print("Number of columns left after imputation:", len(df.columns))

        # Print the number of rows and columns
        print(f"Number of rows and columns: {df.shape}")
        return df

In [5]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if col[-1] in ("P", "A")]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def date_expr(df):
        cols = [col for col in df.columns if col[-1] in ("D",)]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def str_expr(df):
        cols = [col for col in df.columns if col[-1] in ("M",)]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def other_expr(df):
        cols = [col for col in df.columns if col[-1] in ("T", "L")]
        
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max
    
    @staticmethod
    def count_expr(df):
        cols = [col for col in df.columns if "num_group" in col]

        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]

        return expr_max

    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [6]:
def read_file(path, depth=None):
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    
    if depth in [1, 2]:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    
    return df

def read_files(regex_path, depth=None):
    chunks = []
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        
        if depth in [1, 2]:
            df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
        
        chunks.append(df)
        
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])

    return df
    

In [7]:
from datetime import datetime

def calculate_age_from_year(year):
    if year is None:
        return None
    current_year = datetime.today().year
    return current_year - year

def combine_education_levels(edu1, edu2):
    # Example combination logic: take the maximum education level
    # Modify this logic based on your specific needs and data characteristics
    if edu1 is None and edu2 is None:
        return None
    elif edu1 is None:
        return edu2
    elif edu2 is None:
        return edu1
    else:
        return max(edu1, edu2)

def feature_eng(df_base, depth_0, depth_1, depth_2):
    # Add basic date features
    df_base = df_base.with_columns(
        pl.col("date_decision").dt.month().alias("month_decision"),
        pl.col("date_decision").dt.weekday().alias("weekday_decision"),
    )
    
    # Join additional dataframes
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
        
    # Feature engineering: Sum of credit limits if they exist
    credit_limit_cols = ["credlmt_1052A", "credlmt_3940954A", "credlmt_935A"]
    if all(col_name in df_base.columns for col_name in credit_limit_cols):
        df_base = df_base.with_columns(
            (pl.col(credit_limit_cols).sum()).alias("total_credit_limit")
        )
    
    # Feature engineering: Deposit Ratio
    deposit_incoming_col = "amtdepositincoming_4809444A"
    deposit_outgoing_col = "amtdepositoutgoing_4809442A"
    
    if deposit_incoming_col in df_base.columns and deposit_outgoing_col in df_base.columns:
        df_base = df_base.with_columns(
            (pl.col(deposit_incoming_col) / pl.col(deposit_outgoing_col)).alias("deposit_ratio")
        )
    
    # Feature engineering: Date of birth related features (calculating age only)
    date_of_birth_cols = ["dateofbirth_337D", "dateofbirth_342D"]
    
    for col in date_of_birth_cols:
        if col in df_base.columns:
            df_base = df_base.with_columns(
                pl.col(col).cast(pl.Date).dt.year().apply(calculate_age_from_year, return_dtype=pl.Int64).alias(f"{col}_age")
            )
    
    # Feature engineering: Combine credit bureau queries
    query_cols = ["days120_123L", "days180_256L", "days30_165L", "days360_512L", "days90_310L"]
    
    if all(col in df_base.columns for col in query_cols):
        df_base = df_base.with_columns(
            sum([pl.col(col) for col in query_cols]).alias("total_queries")
        )
    
    # Feature engineering: Aggregate installment payment percentages
    early_payment_col = "pctinstlsallpaidearl3d_427L"
    late_payment_cols = [
        "pctinstlsallpaidlat10d_839L",
        "pctinstlsallpaidlate1d_3546856L",
        "pctinstlsallpaidlate4d_3546849L",
        "pctinstlsallpaidlate6d_3546844L"
    ]
    
    if early_payment_col in df_base.columns:
        df_base = df_base.with_columns(
            pl.col(early_payment_col).alias("total_early_payments")
        )
    
    if all(col in df_base.columns for col in late_payment_cols):
        df_base = df_base.with_columns(
            sum([pl.col(col) for col in late_payment_cols]).alias("total_late_payments")
        )
    
    # Feature engineering: Combine education levels
    edu1_col = "education_88M"
    edu2_col = "education_927M"
    
    if edu1_col in df_base.columns and edu2_col in df_base.columns:
        df_base = df_base.with_columns(
            pl.struct([pl.col(edu1_col), pl.col(edu2_col)]).apply(lambda row: combine_education_levels(row[edu1_col], row[edu2_col]), return_dtype=pl.Utf8).alias("combined_education")
        )
    
    # Handle dates if necessary
    df_base = df_base.pipe(Pipeline.handle_dates)
    
    # Check if the total_credit_limit feature is included
    if "total_credit_limit" in df_base.columns:
        print("total_credit_limit feature is included.")
    else:
        print("total_credit_limit feature is not included.")
        
    # Check if the deposit_ratio feature is included
    if "deposit_ratio" in df_base.columns:
        print("deposit_ratio feature is included.")
    else:
        print("deposit_ratio feature is not included.")
        
    # Check if the date of birth age features are included
    for col in date_of_birth_cols:
        if f"{col}_age" in df_base.columns:
            print(f"{col}_age feature is included.")
        else:
            print(f"{col}_age feature is not included.")
    
    # Check if the credit bureau queries feature is included
    if "total_queries" in df_base.columns:
        print("total_queries feature is included.")
    else:
        print("total_queries feature is not included.")
        
    # Check if the installment payment percentage features are included
    if "total_early_payments" in df_base.columns:
        print("total_early_payments feature is included.")
    else:
        print("total_early_payments feature is not included.")
    
    if "total_late_payments" in df_base.columns:
        print("total_late_payments feature is included.")
    else:
        print("total_late_payments feature is not included.")
        
    # Check if the combined education feature is included
    if "combined_education" in df_base.columns:
        print("combined_education feature is included.")
    else:
        print("combined_education feature is not included.")
        
    return df_base


In [8]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    
    return df_data, cat_cols

In [9]:

##ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")
##TRAIN_DIR       = ROOT / "parquet_files" / "train"
##TEST_DIR        = ROOT / "parquet_files" / "test"

from pathlib import Path
TRAIN_DIR = Path("../parquet_files/train")
TEST_DIR = Path("../parquet_files/test")


In [10]:
data_store = {
    "df_base": read_file(TRAIN_DIR / "train_base.parquet"),
    "depth_0": [
        read_file(TRAIN_DIR / "train_static_cb_0.parquet"),
        read_files(TRAIN_DIR / "train_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TRAIN_DIR / "train_applprev_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_a_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_tax_registry_c_1.parquet", 1),
        read_files(TRAIN_DIR / "train_credit_bureau_a_1_*.parquet", 1),
        read_file(TRAIN_DIR / "train_credit_bureau_b_1.parquet", 1),
        read_file(TRAIN_DIR / "train_other_1.parquet", 1),
        read_file(TRAIN_DIR / "train_person_1.parquet", 1),
        read_file(TRAIN_DIR / "train_deposit_1.parquet", 1),
        read_file(TRAIN_DIR / "train_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TRAIN_DIR / "train_credit_bureau_b_2.parquet", 2),
        read_files(TRAIN_DIR / "train_credit_bureau_a_2_*.parquet", 2),
    ]
}

In [11]:
df_train = feature_eng(**data_store)

print("train data shape:\t", df_train.shape)

C:\Users\Darren Chang\AppData\Local\Temp\ipykernel_15800\3423818613.py:44: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(col).cast(pl.Date).dt.year().apply(calculate_age_from_year, return_dtype=pl.Int64).alias(f"{col}_age")
C:\Users\Darren Chang\AppData\Local\Temp\ipykernel_15800\3423818613.py:44: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(col).cast(pl.Date).dt.year().apply(calculate_age_from_year, return_dtype=pl.Int64).alias(f"{col}_age")


total_credit_limit feature is not included.
deposit_ratio feature is not included.
dateofbirth_337D_age feature is included.
dateofbirth_342D_age feature is included.
total_queries feature is included.
total_early_payments feature is included.
total_late_payments feature is included.
train data shape:	 (1526659, 477)


In [12]:
data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
    ],
    "depth_2": [
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
    ]
}

In [13]:
df_test = feature_eng(**data_store)

print("test data shape:\t", df_test.shape)

total_credit_limit feature is not included.
deposit_ratio feature is not included.
dateofbirth_337D_age feature is included.
dateofbirth_342D_age feature is included.
total_queries feature is included.
total_early_payments feature is included.
total_late_payments feature is included.
test data shape:	 (10, 476)


C:\Users\Darren Chang\AppData\Local\Temp\ipykernel_15800\3423818613.py:44: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(col).cast(pl.Date).dt.year().apply(calculate_age_from_year, return_dtype=pl.Int64).alias(f"{col}_age")
C:\Users\Darren Chang\AppData\Local\Temp\ipykernel_15800\3423818613.py:44: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col(col).cast(pl.Date).dt.year().apply(calculate_age_from_year, return_dtype=pl.Int64).alias(f"{col}_age")


In [14]:
# Convert train data to pandas and filter
df_train, cat_cols = to_pandas(df_train)
df_train_filtered = df_train.pipe(Pipeline.filter_cols)

# Convert test data to pandas and filter
df_test, cat_cols = to_pandas(df_test, cat_cols)
df_test_filtered = df_test.pipe(Pipeline.filter_cols)


# Print the number of columns left
print("Number of columns left before filtering:", len(df_train_filtered.columns))
print("Number of columns left before filtering:", len(df_test_filtered.columns))


# Ensure that both train and test data have the same columns except for the target attribute
common_columns = list(set(df_train_filtered.columns).intersection(set(df_test_filtered.columns)))

# Add specific columns to the common columns list if they are not already present
columns_to_keep = ['case_id', 'WEEK_NUM', 'month_decision', 'weekday_decision']
for col in columns_to_keep:
    if col not in common_columns:
        common_columns.append(col)

# Preserve the original order of columns in df_train_filtered
common_columns = [col for col in df_train_filtered.columns if col in common_columns]

# Add 'target' column to train data
df_train_filtered = df_train_filtered[common_columns + ['target']]

# Filter test data to include only common columns
df_test_filtered = df_test_filtered[common_columns]

# Print the number of columns left
print("Number of columns left after filtering:", len(df_train_filtered.columns))
print("Number of columns left after filtering:", len(df_test_filtered.columns))




Null values in 'case_id': 0
Null values in 'WEEK_NUM': 0
Null values in 'target': 0
Null values in 'month_decision': 0
Null values in 'weekday_decision': 0
Null values in 'dateofbirth_337D': 140968
Null values in 'days120_123L': 140968
Null values in 'days180_256L': 140968
Null values in 'days30_165L': 140968
Null values in 'days360_512L': 140968
Null values in 'days90_310L': 140968
Null values in 'description_5085714M': 26183
Null values in 'education_1103M': 26183
Null values in 'education_88M': 26183
Null values in 'firstquarter_103L': 140968
Null values in 'fourthquarter_440L': 140968
Null values in 'maritalst_385M': 26183
Null values in 'maritalst_893M': 26183
Null values in 'numberofqueries_373L': 140968
Null values in 'secondquarter_766L': 140968
Null values in 'thirdquarter_1082L': 140968
Null values in 'actualdpdtolerance_344P': 418178
Null values in 'amtinstpaidbefduel24m_4187115A': 561124
Null values in 'annuity_780A': 0
Null values in 'annuitynextmonth_57A': 4
Null values i

In [15]:

del data_store
gc.collect()

0

In [16]:
print("Train is duplicated:\t", df_train_filtered["case_id"].duplicated().any())
print("Train Week Range:\t", (df_train_filtered["WEEK_NUM"].min(), df_train["WEEK_NUM"].max()))

print()

print("Test is duplicated:\t", df_test_filtered["case_id"].duplicated().any())
print("Test Week Range:\t", (df_test_filtered["WEEK_NUM"].min(), df_test["WEEK_NUM"].max()))

Train is duplicated:	 False
Train Week Range:	 (0, 91)

Test is duplicated:	 False
Test Week Range:	 (100, 100)


In [17]:
# sns.lineplot(
#     data=df_train,
#     x="WEEK_NUM",
#     y="target",
# )
# plt.show()

In [18]:
X = df_train_filtered.drop(columns=["target", "case_id", "WEEK_NUM"])
y = df_train_filtered["target"]
weeks = df_train_filtered["WEEK_NUM"]

cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "learning_rate": 0.05,
    "n_estimators": 1000,
    "colsample_bytree": 0.8, 
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    
    "class_weight": "balanced",
    #"device": "gpu",
}

fitted_models = []

for idx_train, idx_valid in cv.split(X, y, groups=weeks):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)]
    )

    fitted_models.append(model)

model = VotingModel(fitted_models)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.813154
[200]	valid_0's auc: 0.815642
Early stopping, best iteration is:
[173]	valid_0's auc: 0.81608
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.816827
[200]	valid_0's auc: 0.818684
Early stopping, best iteration is:
[153]	valid_0's auc: 0.81929
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.817692
[200]	valid_0's auc: 0.820907
Early stopping, best iteration is:
[158]	valid_0's auc: 0.82129
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.817996
[200]	valid_0's auc: 0.820589
Early stopping, best iteration is:
[142]	valid_0's auc: 0.820836
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.812999
[200]	valid_0's auc: 0.815703
Early stopping, best iteration is:
[165]	valid_0's auc: 0.81618


In [19]:
X_test = df_test_filtered.drop(columns=["WEEK_NUM"])
X_test = X_test.set_index("case_id")

y_pred = pd.Series(model.predict_proba(X_test)[:, 1], index=X_test.index)

In [20]:
df_subm = pd.read_csv("sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred

In [21]:
print("Check null: ", df_subm["score"].isnull().any())

df_subm_sorted = df_subm.sort_values(by="score", ascending=False)
top_5_scores = df_subm_sorted.head()
print(top_5_scores)

Check null:  False
            score
case_id          
57569    0.836175
57549    0.423035
57631    0.357585
57633    0.356964
57552    0.342152


In [22]:
df_subm.to_csv("submission.csv")